In [ ]:
# Imports
from dotenv import load_dotenv
from json import dump
from pathlib import Path
from sklearn.metrics import classification_report, roc_auc_score
from time import time
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
import matplotlib.pyplot as plt
import os
import polars as pl
import torch

In [ ]:
# Load environment variables
load_dotenv()

# Get the directory of the current file
__dir__ = Path(os.path.abspath(""))
"""
The directory of the current file
"""

# Load environment variables
DATASET_NAME = os.environ["DATASET_NAME"]
"""
Dataset name
"""

CLASSIFIER_MODEL_NAME = os.environ["CLASSIFIER_MODEL_NAME"]
"""
Classifier model name
"""

# Create the output directory
OUTPUT_DIRECTORY = __dir__ / f"../data/notebooks/classifier-transformer/{DATASET_NAME.replace("/", "-")}/{CLASSIFIER_MODEL_NAME.replace("/", "-")}"
OUTPUT_DIRECTORY.mkdir(parents=True, exist_ok=True)

In [ ]:
# Load the tokenizer and classifier models
dtype = torch.bfloat16
tokenizer = AutoTokenizer.from_pretrained(CLASSIFIER_MODEL_NAME)
llm_model = AutoModelForSequenceClassification.from_pretrained(CLASSIFIER_MODEL_NAME, dtype=dtype)

In [ ]:
# Load the prepared dataset
test_df = pl.read_parquet(__dir__ / f"../data/notebooks/prepare-datasets/{DATASET_NAME.replace("/", "-")}/test.parquet")

In [ ]:
# Create the classifier pipeline
llm_classifier = pipeline(
  "text-classification",
  model=llm_model,
  tokenizer=tokenizer,
  truncation=True,
  max_length=512,
  device=torch.device("cuda" if torch.cuda.is_available() else "cpu"),
)

In [ ]:
# Benchmark the classifier
start = time()
raw_y_probabilities = test_df["text"].map_elements(llm_classifier)
end = time()

elapsed_test_classify = end - start

In [ ]:
# Convert per-class Y predictions to malicious prediction probabilities
y_probabilities = [
  1 - prediction[0]["score"] if prediction[0]["label"] == llm_model.config.id2label[0] else prediction[0]["score"]
  for prediction in raw_y_probabilities
]

In [ ]:
# Convert Y probabilities to labels
y_predictions = [
    0 if probability < 0.5 else 1
    for probability in y_probabilities
]

In [ ]:
# Get the Y actual labels
y_actual = test_df["label"].to_list()

In [ ]:
# Save the results
with open(OUTPUT_DIRECTORY / "results.json", "w", encoding="utf-8") as results_file:
  dump({
      "auc": roc_auc_score(y_actual, y_predictions),
      "report": classification_report(y_actual, y_predictions, target_names=["benign", "malicious"], output_dict=True),
      "classify_time": elapsed_test_classify,
      "total_time": elapsed_test_classify,
  }, results_file, indent=2)

In [ ]:
# COLUMNS = ["text", "label", "predicted_label", "predicted_malicious_probability"]
# SAMPLE = 50

# # Save some randomly-selected true positives, true negatives, false positives, and false negatives
# results_df = test_df.with_columns(
#     pl.Series("predicted_label", y_predictions),
#     pl.Series(
#         "predicted_malicious_probability",
#         y_probabilities,
#     ),
# )
# tp_df = results_df.filter((pl.col("label") == 1) & (pl.col("predicted_label") == 1))
# tn_df = results_df.filter((pl.col("label") == 0) & (pl.col("predicted_label") == 0))
# fp_df = results_df.filter((pl.col("label") == 0) & (pl.col("predicted_label") == 1))
# fn_df = results_df.filter((pl.col("label") == 1) & (pl.col("predicted_label") == 0))

# pl.concat([df.sample(SAMPLE) for df in [tp_df, tn_df, fp_df, fn_df]]).sample(
#     n=4 * SAMPLE,
#     shuffle=True
# ).select(COLUMNS).write_csv(OUTPUT_DIRECTORY / "samples.csv")

In [ ]:
# Create an overlapping histogram of the prediction probabilities for each class
true_benign_probabilities = [probability for i, probability in enumerate(y_probabilities) if y_actual[i] == 0]
true_malicious_probabilities = [probability for i, probability in enumerate(y_probabilities) if y_actual[i] == 1]

plt.hist(true_benign_probabilities, bins=25, alpha=0.5, label="Benign", color="green")
plt.hist(true_malicious_probabilities, bins=25, alpha=0.5, label="Malicious", color="red")

plt.xlim(0, 1)
plt.xlabel("Predicted Probability of Being Malicious")
plt.ylabel("Frequency")
plt.title("Transformer Classifier Ground Truth Class Probability Distribution")

plt.legend()
plt.savefig(OUTPUT_DIRECTORY / "probability-distribution.png", dpi=600)